## Importing Necessary Libraries

In [2]:
import selenium
import requests
import re
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.support.wait import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.chrome.options import Options
import time
import pandas as pd
import numpy as np


## Starting Browser Instance

In [3]:
#disabling notifications and starting maximised for better presentability
options = Options()
options.add_experimental_option("prefs", { 
    "profile.default_content_setting_values.notifications": 1 
})

#Starting browser instance
driver = webdriver.Chrome(options = options, service=Service(ChromeDriverManager().install()))
driver.get('https://www.amazon.in/s?rh=n%3A6612025031&fs=true&ref=lp_6612025031_sar')
time.sleep(5)#waiting for page to load
driver.maximize_window()
#including out of stock
WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.XPATH, '//*[@id="p_n_availability/1318485031"]/span/a/span'))).click()




## Performing The Task

In [4]:
#Creating DataFrame
productdf = pd.DataFrame(columns = ['Name', 'Price', 'Rating', 'Seller'])

In [5]:
product_index = 1001
while True:
    products = driver.find_elements(By.XPATH, "//a[@class = 'a-link-normal s-underline-text s-underline-link-text s-link-style a-text-normal']")
    for product in products:
        #opening product pages
        product.click()
    time.sleep(3)
    while len(driver.window_handles) > 1:
        #switiching tab to the product page
        current_tab = driver.window_handles[1]
        driver.switch_to.window(current_tab)
        #extracting title
        try:
            Title = WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.XPATH, "//span[@id = 'productTitle']")))
            Title = Title.text
        except:
            pass
            
        #extracting Rating
        Rating = WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.XPATH, "//span[@id = 'acrPopover']/span/a/span")))
        Rating = float(Rating.text)
        
        #checking for unavailability
        if WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.XPATH, '//*[@id="outOfStock"]/div/div[1]/span[1]'))).text == 'Currently unavailable.':
            Price = np.NaN
            Seller = np.NaN
            continue
        else:

            #extracting price
            try:
                price_decimal = WebDriverWait(driver, 5).until(EC.presence_of_element_located((By.XPATH, '//*[@id = "corePrice_feature_div"]/div/div/span[1]/span[2]/span[2]')))
                Price = int(price_decimal.text.replace(',',''))
            except:

                WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.XPATH, "//span[@id = 'buybox-see-all-buying-choices']/span/a"))).click()
                time.sleep(2)
                Price = (WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.XPATH, '//span[@id="aod-price-1"]/div[1]/span[2]/span[2]/span[2]'))).text.replace(',',''))
                Price = int(Price)
                driver.back()

    
            #extracting Seller
            try:
                Seller = (WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.XPATH, '//div[@id="merchant-info"]/a[1]/span'))).text)

            except:
                WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.XPATH, "//span[@id = 'buybox-see-all-buying-choices']/span/a"))).click()
                Seller = WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.XPATH, "//div[@id = 'aod-offer-soldBy']/div/div[1]/a"))).text
                driver.back()

            productdf.loc[product_index] = [Title, Price, Rating, Seller]
            product_index += 1
            driver.close()
    #coming back to main page
    try:
        driver.switch_to.window(driver.window_handles[0])
        #checkking if next page exists
        WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.XPATH, '//*[@id="search"]/div[1]/div[1]/div/span[1]/div[1]/div[35]/div/div/span/a[3]'))).click()
    except:
        break
        


In [6]:
display(productdf)

,Name,Price,Rating,Seller
1001,URBN 10000 mAh 20W Super Fast Charging Ultra C...,1499,4.1,Cocoblu Retail
1002,"Redmi 20000mAh Li-Polymer Power Bank, USB Type...",2049,4.2,Cocoblu Retail
1003,URBN 20000 mAh Premium Nano Power Bank | 22.5W...,2499,4.1,Cocoblu Retail
1004,URBN 20000 mAh Lithium_Polymer 22.5W Super Fas...,1499,3.9,Cocoblu Retail
1005,Amazon Basics 20000mAh 12W Lithium-Polymer Pow...,1219,3.8,Appario Retail Private Ltd
1006,URBN 10000 mAh Lithium_Polymer 22.5W Super Fas...,1199,3.9,Cocoblu Retail
1007,Amazon Basics 5000mAh Li-Polymer Wireless Powe...,1099,4.3,Appario Retail Private Ltd
1008,Amazon Basics 20000mAh 22.5W Lithium-Polymer P...,1579,3.8,Appario Retail Private Ltd
1009,URBN 20000 mAh Premium Black Edition Nano Powe...,2499,4.1,Cocoblu Retail
1010,Ambrane 10000mAh Magnetic Wireless Power Bank ...,1899,2.9,Cocoblu Retail


In [7]:
#transfering to csv file
productdf.to_csv('Data.csv')

In [65]:
# #finding products
# product_index = 1001
# while True:
#     products = driver.find_elements(By.XPATH, "//a[@class = 'a-link-normal s-underline-text s-underline-link-text s-link-style a-text-normal']")
#     for product in products:
#         #opening product pages
#         product.click()
#         current_tab = driver.window_handles[1]
#         driver.switch_to.window(current_tab)
#         try:
#             Title = WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.XPATH, "//span[@id = 'productTitle']")))
#             Title = Title.text
#         except:
#             pass

#         try:
#             price_decimal = WebDriverWait(driver, 5).until(EC.presence_of_element_located((By.XPATH, '//*[@id = "corePrice_feature_div"]/div/div/span[1]/span[2]/span[2]')))
#             #price_float = WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.XPATH, "//span[@class = 'a-price-fraction']")))
#             #Price = float((f'{price_decimal.text}.{price_float.text}').replace(',',''))#getting the total price as int by adding decimal and whole
#             Price = int(price_decimal.text.replace(',',''))
#         except:

#             WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.XPATH, "//span[@id = 'buybox-see-all-buying-choices']/span/a"))).click()
#             time.sleep(2)
#             Price = (WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.XPATH, '//span[@id="aod-price-1"]/div[1]/span[2]/span[2]/span[2]'))).text.replace(',',''))
#             Price = int(Price)
#             Price = (WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.XPATH, '//*[@id="aod-price-1"]/div[1]/span[2]/span[2]/span[2]'))).text.replace(',',''))

#         Rating = WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.XPATH, "//span[@id = 'acrPopover']/span/a/span")))
#         Rating = float(Rating.text)


#         try:
#             Seller = (WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.XPATH, '//div[@id="merchant-info"]/a[1]/span'))).text)

#         except:
#             WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.XPATH, "//span[@id = 'buybox-see-all-buying-choices']/span/a"))).click()
#             Seller = WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.XPATH, "//div[@id = 'aod-offer-soldBy']/div/div[1]/a"))).text
#             driver.back()

#         productdf.loc[product_index] = [Title, Price, Rating, Seller]
#         product_index += 1
#         driver.close()
#         driver.switch_to.window(driver.window_handles[0])
        
#     try:    
#         WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.XPATH, '//*[@id="search"]/div[1]/div[1]/div/span[1]/div[1]/div[35]/div/div/span/a[3]'))).click()
#     except:
#         break
        
        


NoSuchWindowException: Message: no such window: target window already closed
from unknown error: web view not found
  (Session info: chrome=120.0.6099.71)
Stacktrace:
	GetHandleVerifier [0x00D66E73+174291]
	(No symbol) [0x00C90AC1]
	(No symbol) [0x009A6FF6]
	(No symbol) [0x0098EFE7]
	(No symbol) [0x009FB53B]
	(No symbol) [0x009FFAED]
	(No symbol) [0x009F6FD3]
	(No symbol) [0x009D1034]
	(No symbol) [0x009D1F8D]
	GetHandleVerifier [0x00E049CC+820268]
	sqlite3_dbdata_init [0x00EC4EBE+652494]
	sqlite3_dbdata_init [0x00EC48D9+650985]
	sqlite3_dbdata_init [0x00EB962C+605244]
	sqlite3_dbdata_init [0x00EC586B+654971]
	(No symbol) [0x00C9FEBC]
	(No symbol) [0x00C98428]
	(No symbol) [0x00C9854D]
	(No symbol) [0x00C85858]
	BaseThreadInitThunk [0x76F27BA9+25]
	RtlInitializeExceptionChain [0x77ADBD2B+107]
	RtlClearBits [0x77ADBCAF+191]


In [66]:
display(productdf)

,Name,Price,Rating,Seller
1001,"Ambrane 50000mAh Powerbank, 20W Fast Charging,...",4499,4.0,Cocoblu Retail
1002,Oboe Silicon Soft Cover Case for Mi Power Bank...,349,4.0,Dealfreez Estore
1003,Ambrane 20000mAh Power Bank with 20W Fast Char...,1599,4.0,Cocoblu Retail
1004,MI Power Bank 3i 20000mAh Lithium Polymer 18W ...,2149,4.2,Cocoblu Retail
1005,"Mi 10000mAH Li-Polymer, Micro-USB and Type C I...",1299,4.2,Cocoblu Retail
1006,"MI 10000mAh Lithium Ion, Lithium Polymer Power...",1699,4.3,Cocoblu Retail
1007,Ambrane 20000mAh Power Bank with 20W Fast Char...,1599,4.0,Cocoblu Retail
1008,"Ambrane 20000mAh Power Bank, 20W Fast Charging...",1599,4.0,Cocoblu Retail
1009,Duracell 20000 MAH Slimmest Power Bank with 1 ...,3099,4.2,Cocoblu Retail
1010,"Ambrane 10000mAh Slim Power Bank, 20W Fast Cha...",999,3.9,Cocoblu Retail
